In [5]:
import pandas as pd
from dbfread import DBF
path = 'D:\\IPMIProject\\master test\\20231223.dbf' # 文件目录
table = DBF(path, encoding='GBK')
df = pd.DataFrame(iter(table))
print(df)

              BMH ZZLLM   ZZLLMC ZZLLKDDM  ZZLLKDMC ZZLLKCBH ZZLLZWH WGYM  \
0       611689959   199  管理类综合能力     6116      西北大学       99       1  204   
1       510296397   101   思想政治理论     6152    西安翻译学院       11      18  204   
2       610397518   101   思想政治理论     6103    西安石油大学       62      26  201   
3       610797013   101   思想政治理论     6107    西安邮电大学       24       6  201   
4       614698996   101   思想政治理论     6146      西京学院       74      28  204   
...           ...   ...      ...      ...       ...      ...     ...  ...   
172883  653398263   101   思想政治理论     6156    咸阳师范学院      105      11  201   
172884  653399631   101   思想政治理论     6149      安康学院      059      20  201   
172885  653399955   101   思想政治理论     6144    宝鸡文理学院      J31      11  204   
172886  653499532   101   思想政治理论     6141    陕西理工大学      144      28  201   
172887  653499807   101   思想政治理论     6134  西安建筑科技大学      302      09  201   

        WGYMC WGYKDDM  ... YWK2KCBH YWK2ZWH BKDWDM        BKDWMC  BKZYDM  \

In [7]:
# 数据预处理尝试

import numpy as np
import pandas as pd

df.columns = df.columns.map(str.lower)
arr = df
# print(arr.head(10))


# 取出dataframe的几列
df1 = arr[["bmh","zzllmc","wgymc","ywk1mc","ywk2mc","bkzydm"]]

# 使用其中的四个属性进行分组
df2 = df1.groupby(["zzllmc","wgymc","ywk1mc","ywk2mc"]).count()
df2.to_excel("classfier.xlsx")


# sub_DF=[arr[arr['wgymc'].isin([i])] for i in np.unique(arr['wgymc'])]
# print(df2.head())
# groupby后的数据要先转为dataframe，再进行索引重建，才可以保存为xlsx
# class_arr = pd.DataFrame(df1.groupby(["zzllmc","wgymc","ywk1mc","ywk2mc"]))
# class_arr.reset_index(inplace=True)
# class_arr.to_excel("res.xlsx")

In [8]:
# 按照外国语考点代码拆分数据

import numpy as np
import pandas as pd

# arr = pd.read_excel("tmp.xlsx")
list1 = arr['wgykddm'].unique()
groups = arr.groupby("wgykddm")
for i in arr['wgykddm'].unique():
    groups.get_group(i).to_excel(str(i)+".xlsx")
# print(groups['wgykddm'])
    # 检查数量是否相等
# print(arr.shape)#(16383,32)
# sum = 0;
# for i in arr['wgykddm'].unique():
#     df1 = pd.read_excel(str(i)+".xlsx")
#     sum += df1.shape[0]

# print(sum)#16383

In [9]:
# 统计每个考点教室所能容纳的人数
groups1 = pd.read_excel("考场容量表.xls").groupby("zzllkddm")
print(groups1.apply(lambda x:x))
print(groups1.get_group(6103)["renshu"].tolist())

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
               zzllkddm zzllkcbh  renshu
zzllkddm                                
6103     0         6103        1      30
         1         6103       10      30
         2         6103      100      30
         3         6103      101      30
         4         6103      102      30
...                 ...      ...     ...
6160     5420      6160      103      30
         5421      6160      104      30
         5422      6160      105      30
         5423      6160      106      30
         5424      6160      107      16

[5425 rows x 3 columns]
[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 29, 30, 30, 30, 31, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 

In [10]:
# 统计每个考点有多少专业，以及每个专业有多少人，参考属性有["zzllmc","wgymc","ywk1mc","ywk2mc","bkdwdm"]

for i in arr['wgykddm'].unique():
    df1 = pd.read_excel(str(i)+".xlsx")

    # 使用其中的四个属性进行分组
    df2 = df1.groupby(["zzllmc","wgymc","ywk1mc","ywk2mc","bkdwdm"]).count()
    df2.to_excel("classfier"+str(i)+".xlsx")

# 结果里面bmh表示的是每个专业的人数

In [11]:
# 数据预处理、提取出每个考点，每个专业的人数
sums = 0
print(list1)
for i in list1:
    # .fillna(method="ffill")向下填充
    res = pd.read_excel("classfier"+str(i)+".xlsx").fillna(method="ffill")
    # dataframe按各专业人数一列排序，inplace原地排序，ascending升序
    # res.sort_values(by="bmh",inplace=True,ascending=False)
    # 取出各专业人数一列转为array
    # 若存在控制需要填充
    # res = res.fillna(res['bmh'].mean())
    StuNum = res[['bmh','zzllmc','wgymc','ywk1mc','ywk2mc',"bkdwdm"]]
    np.savetxt("classfier"+str(i)+".txt",StuNum,fmt="%d,%s,%s,%s,%s,%s",delimiter=" ")
    sums += res['bmh'].sum()

print(sums)

# classfierX.excel中的前四列可以映射到对应的专业

['6116' '6152' '6103' '6107' '6146' '6144' '6143' '6139' '6117' '6153'
 '6118' '6160' '6141' '6114' '6113' '6124' '6134' '6105' '6136' '6149'
 '6159' '6157' '6151' '6115' '6112' '6111' '6156' '6133' '6148' '6140'
 '6125' '6154' '6147' '6150' '6145' '6127' '6138' '6128' '6122' '6129'
 '6130' '6131' '6142' '6158']
172888


In [12]:
def Sum_matrix(matrix):
    sum=0
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            sum+=matrix[i][j]
    return sum

In [18]:
import pandas as pd
pd.options.display.max_rows = None
# 分组算法
# pd.set_option('display.max_rows',200)
# StuNum = np.loadtxt("classfier6127.txt")
aa=pd.read_csv("classfier6105.txt",encoding='gb2312',header=None)
aa.columns=['num','s1','s2','s3','s4','s5']
# 对txt文件的数据排序，方便后面索引对应
print(aa.sort_values(by='num',axis=0).reset_index(drop=True))
StuNum = pd.read_csv("classfier6105.txt",encoding='gb2312',header=None).iloc[:,0].tolist()

# 设置参数：教室最多可容纳人数、考场允许最大专业数量
max_student=groups1.get_group(6105)["renshu"].tolist()
max_subject=5
# 存放分组结果
groups = []

# 这里还没有考虑取余的情况
for i in range(len(StuNum)):
    StuNum[i] = StuNum[i] % 30

StuNum.sort()
print(len(StuNum))
start = 0
end = len(StuNum)-1
classroom = 0
while(start<end and classroom<len(max_student)):
    class_subject = []#用来存储人数对应的索引，输出的就是这个，和上面txt文件的输出对应
    class_sum = StuNum[end]#每次循环初始，让sum等于尾指针指向的数
    subject = 1
    class_subject.append(end)
    # 人数不超过教室容量、专业不超过教室容量、专业还未分完
    while(class_sum<max_student[classroom] and subject<max_subject and start<end):
        class_sum = class_sum + StuNum[start]
        class_subject.append(start)
        start = start+1
        subject = subject+1
    end = end-1
    classroom+=1
    groups.append(class_subject)

print(groups)
print(len(groups))
# 下面的数字是不同袋子的编号

      num       s1      s2                    s3                       s4  \
0       1   思想政治理论    二外日语                  基础英语                  英语写作与翻译   
1       1   思想政治理论   英语（二）                  体育综合                        无   
2       1   思想政治理论   英语（二）                  体育综合                        无   
3       1   思想政治理论   英语（二）                  体育综合                        无   
4       1   思想政治理论   英语（二）                  体育综合                        无   
5       1   思想政治理论   英语（二）                  体育综合                        无   
6       1   思想政治理论   英语（二）                  体育综合                        无   
7       1   思想政治理论   英语（二）                  体育综合                        无   
8       1   思想政治理论   英语（二）                  体育综合                        无   
9       1   思想政治理论   英语（二）                  体育综合                        无   
10      1   思想政治理论   英语（二）                  体育综合                        无   
11      1   思想政治理论   英语（二）                  体育综合                        无   

In [9]:
# 分组算法
import os
list1 = []
names = os.listdir("./")
for name in names:
    if name.endswith('.txt'):
        list1.append(name)
print(list1)

summ=0
for j in list1:
    # 读取文件并定位到人数一列
    if j=='classfier6127.txt':
        print(j)
        # header=None表示没有列标签
    StuNum = pd.read_csv(j,encoding='gb2312',header=None).iloc[:,0]
    # print(StuNum)

    # 设置参数：教室最多可容纳人数、考场允许最大专业数量
    max_student=30
    max_subject=5
    groups = []
    dic = {}
    # 建立原始值 和 余数的映射
    for i in range(len(StuNum)):
        dic[StuNum[i]] = StuNum[i] % 30

    # 保存字典映射关系
    StuNum = []
    with open('params.txt', 'w') as f:
        for key, value in dic.items():
            f.write(str(key))
            f.write(': ')
            f.write(str(value))
            f.write('\n')
            StuNum.append(value)


    # 对余数进行排序
    # dic = sorted(dic.items(),key=lambda x:x[1],reverse=False)
    StuNum.sort()
    # print(len(StuNum))
    start = 0
    end = len(StuNum)-1
    while(start<end):
        class_subject = []
        class_sum = StuNum[end]
        subject = 1
        class_subject.append(end)
        # ===================================================文件6127在这里出了问题
        while(class_sum<max_student and subject<max_subject):
            class_sum = class_sum + StuNum[start]
            class_subject.append(start)
            start = start+1
            subject = subject+1
        end = end-1
        groups.append(class_subject)

    print(groups)
    print(j)
    summ=summ+Sum_matrix(groups)
    print("\n")
print(summ)
    # 下面的数字是不同袋子的编号

['classfier6103.txt', 'classfier6105.txt', 'classfier6107.txt', 'classfier6111.txt', 'classfier6112.txt', 'classfier6113.txt', 'classfier6114.txt', 'classfier6115.txt', 'classfier6116.txt', 'classfier6117.txt', 'classfier6118.txt', 'classfier6122.txt', 'classfier6124.txt', 'classfier6125.txt', 'classfier6127.txt', 'classfier6128.txt', 'classfier6129.txt', 'classfier6133.txt', 'classfier6134.txt', 'classfier6136.txt', 'classfier6138.txt', 'classfier6139.txt', 'classfier6140.txt', 'classfier6141.txt', 'classfier6143.txt', 'classfier6144.txt', 'classfier6145.txt', 'classfier6146.txt', 'classfier6147.txt', 'classfier6148.txt', 'classfier6149.txt', 'classfier6150.txt', 'classfier6151.txt', 'classfier6152.txt', 'classfier6153.txt', 'classfier6154.txt', 'classfier6156.txt', 'classfier6157.txt', 'classfier6159.txt', 'classfier6160.txt', 'params.txt']
[[10, 0, 1, 2], [9, 3, 4, 5, 6], [8, 7, 8]]
classfier6103.txt


[[7, 0, 1, 2, 3], [6, 4, 5, 6, 7]]
classfier6105.txt


[[9, 0], [8, 1, 2, 3, 4], 

IndexError: list index out of range